<span type="title">单件模式</span> | <span type="update">2018-09-02</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章介绍单件模式</p></span>

# 单件模式

在一些情况下，我们需要一个类只保留一个实例，比如线程池、注册表等资源敏感型组件中。这种需求可以使用单件设计模式。单件的可能替代有：全局变量、静态变量，但是这些方式都或多或少有一些问题，尤其是在工程和多线程方便。

## 单线程版本

一个典型的单线程单件模式要考虑到使用代码人员的愚蠢问题：

```java
public class Singleton {
    private static Singleton instance;
    private Singleton() {}
    public static Singleton getInstance() {
        if (instance == null) {
            instance = new Singleton();
        } return instance;
    }
}
```

这种方式是典型的单件模式，使用 private 避免了公开构造，getInstance静态方法可以直接返回一个实例，并且当多次调用时，始终返回一个实例。这种设计的缺点在于 private 不能继承，如果改为 protected，那么又违反了单件的初衷。

你可能想说 instance 变量直接 `instance = new Singleton()` 更方便，不用写条件语句判断，但是这样的话，构造对象的压力很大（也就是Singleton.class 加载时的压力很大）。

## 多线程版本

除此之外，这个经典模式的问题时：在多线程情况下可能产生多个instance。解决办法之一是：`public static synchronized Singleton getInstance() {}` 改写成这种同步的方法即可解决多线程问题。但是，这对性能有影响（100倍左右），如果有性能需求，则可以考虑使用急切模式，也就是直接使用全局变量先初始化 static 的 instance 域，这样JVM加载Singleton类的压力会变大。当然，如果这不是瓶颈的话，其实无所谓，使用同步，或者全局变量均可解决多线程问题。

但是，最好的解决办法是，使用双重检查加锁：

```java
public class Singleton {
    private volatile static Singleton instance;
    private Singleton() {}
    public static Singleton getInstance() {
        if (instance == null) {
            synchronized (Singleton.class) {
                if (instance == null) {
                    instance = new Singleton();
                }
            }
        } return instance;
    }
}
```
这种方法仅在首次调用 Singletion 类时进入 synchronized 模式，创建实例，此外，均直接返回实例，避免了频繁调用 getInstance 的性能问题。注意，这里的 instance 全局变量使用了 `volatile` 关键字，这个关键字能够确保多线程中互相得知这个变量的变化情况。synchronized 则是确保在同一时间只有一个线程进入其内部代码块。